In [1]:
import numpy as np
import pandas as pd
# from plotly.subplots import make_subplots
# import plotly.graph_objects as go
# import plotly.express as px
from scipy.integrate import solve_ivp 
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from mpl_toolkits import mplot3d
import string
import networkx as nx

# Simul params

In [ ]:
# Number of individuals in the community
N = 10000

# Number of communities and edges added per epoch
M = 100

# Start and End epochs
t0, tf = 0, 3000

# Number of timesteps
Nt = 3000

# Time series
t = np.linspace(t0, tf, Nt + 1)

# Fraction of patches where we will apply the test kits
p = 0.4

# Num of patches where we will apply the test kits
l1 = int(p*M)            # For p fraction
l2 = M-l1                # For the reverse case

# Differential equation params
sigma = 0.1
a0 = 0.01
a1 = 0.0001
gamma = 0.07
zeta = 0.02
chi = 0.1

# Setup for the beta slider
beta_min = 0
beta_max = 0.06

beta_steps = M - 1

# Coupling strength steps
epsilon_steps = M

# The randomisation for exposed population will be upto max_exposed
max_exposed = 300

# Generating all the lists
epsilon_list = np.logspace(-5, 1, epsilon_steps, base=10)

In [ ]:
# Making a complete graph with 4 Nodes
adj_mat = np.full((M, M), 1, dtype="int").reshape(M, M)
# graph = nx.erdos_renyi_graph(M, 0.3, seed=None, directed=False)
# adj_mat = nx.to_numpy_array(graph).reshape(M, M)

# Making the degree list for each node (di = M-1 here)
degree_list = np.sum(adj_mat, axis=0).reshape(M, 1)
avg_degree=np.mean(degree_list)

# Dividing the adjacency matrix by rowsum
row_sums= np.sum(adj_mat, axis=1)
adj_mat_normalised = (adj_mat/row_sums[:,np.newaxis]).reshape(M, M)

In [ ]:
def get_degree_counts(n : int, adj_mat):
    freq_map = np.zeros(n) # degree -> count
    for i in range(n):
        num_neighbours = int(np.sum(adj_mat[i]))
        freq_map[num_neighbours] += 1
    return np.array(freq_map)

In [ ]:
# Returns a probabiltity distribution
# that is based on degree, pi = di/total(di) 
def degree_centrality(adj_mat, patches_to_apply):
    # Get degree_list
    degree_list = np.sum(adj_mat, axis=0, dtype="float")
    # Replace with zero in the range [patches_to_apply, len(adj_list))
    np.put(degree_list, np.arange(patches_to_apply, M), 0)

    return degree_list / np.sum(degree_list, dtype="float")

In [ ]:
# Returns a uniform probability distribution
def const_prob(adj_mat, patches_to_apply):
    m = int(len(adj_mat)) # Number of communities

    # each node as probability as 1/patches_to_apply
    retval = np.full(m, 1, dtype="float") / float(patches_to_apply)

    # Replace with zero in the range [patches_to_apply, len(adj_list))
    np.put(retval, np.arange(patches_to_apply, M), 0)

    return retval

In [ ]:
# Returns node wise distribution strategy for test kits
def g_val(a1, K, prob_distri_test_kit):
    return a1 * K * prob_distri_test_kit

In [ ]:
def migration_term(adj_mat, adj_mat_normalised, compartment):
    # Getting the Amn x Sm term
    retval = np.dot(adj_mat_normalised, compartment).reshape(int(len(compartment)), 1)
    
    # Returning the migration amount
    return (retval - compartment)

In [2]:
def ODE_multi_node_with_testkit(t, X_flat, beta, adj_mat, adj_mat_normalised, degree_list, shape_X, epsilon, num_patches, l, gamma, sigma, zeta, chi, a0, a1):
    # Unraveling the X_flat vector
    X = X_flat.reshape(shape_X) 
    S, E, I, H, R, K = X
    
    # Reshaping into patchesx1 matrix from rank-1 array
    S = S.reshape(num_patches, 1)
    E = E.reshape(num_patches, 1)
    I = I.reshape(num_patches, 1)
    H = H.reshape(num_patches, 1)
    R = R.reshape(num_patches, 1)
    
    # Warning : K is a number NOT a list
    # So, we will turn it into a "fake" list by repeating it num_patches times
    K = K.reshape(num_patches, 1)
    
    # We are using degree centrality based test kit distribution
    g = g_val(a1, K[0], const_prob(adj_mat, l).reshape(num_patches, 1))

    # Reshaping into num_patches x 1 matrix
    g = g.reshape(num_patches, 1)
    
    # Obtaining the total population of each patch
    tot_pop = S + E + I + H + R
    
    # Hospitalization param
    alpha = a0 + g
    
    S_migration = ((epsilon) * migration_term(adj_mat, adj_mat_normalised, S)).reshape(num_patches, 1)
    E_migration = ((epsilon) * migration_term(adj_mat, adj_mat_normalised, E)).reshape(num_patches, 1)
    I_migration = ((epsilon) * migration_term(adj_mat, adj_mat_normalised, I)).reshape(num_patches, 1)
    R_migration = ((epsilon) * migration_term(adj_mat, adj_mat_normalised, R)).reshape(num_patches, 1)
    
    # Find the derivatives list, broadcasting coming in clutch now
    dSdt = (np.divide(-(beta * S * I), tot_pop)) +  S_migration
    dEdt = (np.divide((beta * S * I), tot_pop)) - (sigma * E) + E_migration
    dIdt = (sigma * E) -  (alpha * I) + I_migration
    dHdt = (alpha * I) - (gamma * H)
    dRdt = (gamma * H) + R_migration
    dKdt = (zeta * np.full(num_patches, np.sum(I)).reshape(num_patches, 1)) - (chi * K)
    
    # Reshape the derivatives list for good measure
    dSdt = dSdt.reshape(num_patches, 1)
    dEdt = dEdt.reshape(num_patches, 1)
    dIdt = dIdt.reshape(num_patches, 1)
    dHdt = dHdt.reshape(num_patches, 1)
    dRdt = dRdt.reshape(num_patches, 1)
    dKdt = dKdt.reshape(num_patches, 1)
        
    derivs =  np.array([dSdt, dEdt, dIdt, dHdt, dRdt, dKdt])
    
    return derivs.ravel()

def ODE_multi_node_without_testkit(t, X_flat, beta, adj_mat, adj_mat_normalised, degree_list, shape_X, epsilon, num_patches, l, gamma, sigma, zeta, chi, a0, a1):
    return ODE_multi_node_with_testkit(t, X_flat, beta, adj_mat, adj_mat_normalised, degree_list, shape_X, epsilon, num_patches, l, gamma, sigma, zeta, chi, a0, 0)

ERROR: Error in parse(text = x, srcfile = src): <text>:1:5: unexpected symbol
1: def ODE_multi_node_with_testkit
        ^


In [ ]:
def solver(adj_mat, adj_mat_normalised, degree_list, M, beta, ODE_func, E0, epsilon, num_patches, l):
    # Initial conditions for given beta
    I0, H0, R0, K0 = np.zeros(M), np.zeros(M), np.zeros(M), np.zeros(M)

    # Mandatory reshaping, for safety
    E0 = E0.reshape(M, 1)
    I0 = I0.reshape(M, 1)
    H0 = H0.reshape(M, 1)
    R0 = R0.reshape(M, 1)
    K0 = K0.reshape(M, 1)
    
    # Getting the susceptibles
    # [TBX : This makes N*M susceptibles by broadcasting]
    Total_population = np.full(M, N).reshape(M, 1)
    S0 = Total_population - E0 - I0 - H0 - R0
    S0 = S0.reshape(M, 1)

    # Making the state vector
    X0 = np.array([S0, E0, I0, H0, R0, K0])

    # Ravel the X vector temporarily
    shape_X = X0.shape
    X_flat = X0.ravel()

    # Solving the initial value problem
    solved_ivp = solve_ivp(ODE_func, (t0, tf), X_flat, args=(beta, adj_mat, adj_mat_normalised, degree_list, shape_X, epsilon, num_patches, l, gamma, sigma, zeta, chi, a0, a1), t_eval=t)

    # Returning the raw solved ivp data for plotting
    # solved_ivp.y.reshape(6, M, Nt+1)
    return solved_ivp

## Functions for reduced equations

In [ ]:
def func(mig_array, H, K, alpha, beta, gamma, sigma, zeta, chi):
    S,E,I,R= mig_array
    N_mig_arr= np.sum(mig_array) + H

    mig_array_dot= np.array([
        -beta*S*I / N_mig_arr,
        beta*S*I / N_mig_arr - sigma*E, 
        sigma*E - alpha*I, 
        gamma*H
    ])
    
    return mig_array_dot, alpha*I-gamma*H

In [ ]:
def ODE_for_reduced_eqns(t, X_flat, l1,l2, epsilon, beta, gamma, sigma, zeta, chi, a0, a1):
    mig_array = X_flat[0:8]
    H1= X_flat[8]
    H2=X_flat[9]
    K= X_flat[10]
    
    # Getting the 2*4 migration variables
    mig_array = mig_array.reshape(2,4)
    
    # Getting the 2 communities
    C1 = mig_array[0]
    C2 = mig_array[1]
    
    # Getting alpha for C1 and C2
    alpha_C1 = a0 + a1*K/l1
    alpha_C2 = a0
    
    # Getting derivatives
    C1_dot,H1_dot = func(C1, H1, K, alpha_C1, beta, gamma, sigma, zeta, chi) 
    C2_dot,H2_dot = func(C2, H2, K, alpha_C2, beta, gamma, sigma, zeta, chi)
    
    C1_dot = C1_dot + (l2*epsilon/(avg_degree))*(C2-C1)
    C2_dot = C2_dot + (l1*epsilon/(avg_degree))*(C1-C2)
    
    K_dot = zeta*(l1*C1[2]+l2*C2[2]) - chi*K
    
    X_dot = np.concatenate((C1_dot,C2_dot,H1_dot,H2_dot,K_dot), axis=None)
    
    return X_dot

In [ ]:
# X and Y with testkits
x_with_testkits1 = np.array([])
y_with_testkits1 = np.array([])
y_with_testkits1_mean = np.array([])
y_with_testkits1_max = np.array([])
y_with_testkits1_min = np.array([])

# X and Y without testkits
x_without_testkits1 = np.array([])
y_without_testkits1 = np.array([])
y_without_testkits1_mean = np.array([])
y_without_testkits1_max = np.array([])
y_without_testkits1_min = np.array([])
    
# Arrays for reduced equation solver
x_with_testkitsR1 = np.array([])
x_without_testkitsR1 = np.array([])

for epsilon_value in epsilon_list:
    # initial conditions
    custom_E0 = np.random.randint(1, max_exposed, size=(M,1))
#-------------------Reduced Model-----------------------
    E1,E2= custom_E0[0][0],custom_E0[l1][0]
    # Initial conditions for given epsilon
    X0= np.array([N-E1,E1,0,0,N-E2,E2,0,0,0,0,0])

    X_flat = X0.ravel()

    # Solving the initial value problem
    solved_ivp = solve_ivp(ODE_for_reduced_eqns, (t0, tf), X_flat, args=(l1, M-l1, epsilon_value,  0.03, gamma, sigma, zeta, chi, a0, a1))

    # Obtaining the Infection values for both communities
    I_1 = solved_ivp.y[2]
    I_2 = solved_ivp.y[6]

    # Getting the I_max values from I_vals
    I_max_1 = np.max(I_1)
    I_max_2 = np.max(I_2)
    
    x_with_testkitsR1 = np.append(x_with_testkitsR1, I_max_1/N)
    x_without_testkitsR1 = np.append(x_without_testkitsR1, I_max_2/N)
#--------------------------------------------------------

    # Solving the IVP using our solver function
    solved_ivp = solver(adj_mat, adj_mat_normalised, degree_list, M, 0.03, ODE_multi_node_with_testkit, custom_E0, epsilon_value, M, l1)

    # Obtaining the shape we need to realize the y values
    reshape_vals = (6, M, solved_ivp.y.shape[1])
    
    # Getting the y value reshaped
    jffy = solved_ivp.y.reshape(reshape_vals)[2]

    # Obtaining the Infection values for each communities
    I_vals = (solved_ivp.y.reshape(6, M, reshape_vals[2])[2]).reshape(M, reshape_vals[2])

    # Getting the I_max values from I_vals
    I_max = np.max(I_vals, axis=1).reshape(M, 1)
    
    # Colour list for testkit and testkit-free communities
    color_list = ["#F08080", "#80F080", "#8080F0"]
    
    # x1, y1 for the "with testkit nodes"
    x1 = np.full(M, epsilon_value)[:l1]
    y1 = I_max.T[0][:l1] / N
    marker_color_with_testkits = color_list[0]
    
    y_with_testkits1 = np.append(y_with_testkits1, y1)
    x_with_testkits1 = np.append(x_with_testkits1, x1)
    y_with_testkits1_mean = np.append(y_with_testkits1_mean, [np.sum(y1) / l1])
    y_with_testkits1_max = np.append(y_with_testkits1_max, [np.max(y1)])
    y_with_testkits1_min = np.append(y_with_testkits1_min, [np.min(y1)])
    
    # x2, y2 for the "without testkit nodes"
    x2 = np.full(M, epsilon_value)[l1:-1]
    y2 = I_max.T[0][l1:-1] / N
    marker_color_without_testkits = color_list[1]
    
    y_without_testkits1 = np.append(y_without_testkits1, y2)
    x_without_testkits1 = np.append(x_without_testkits1, x2)
    y_without_testkits1_mean = np.append(y_without_testkits1_mean, [np.sum(y2) / (y2.shape[0])])
    y_without_testkits1_max = np.append(y_without_testkits1_max, [np.max(y2)])
    y_without_testkits1_min = np.append(y_without_testkits1_min, [np.min(y2)])

    # Increment the current epsilon step number
    I_max_vs_epsilon_list = np.zeros(shape=(epsilon_steps, M))

In [ ]:
plt.figure(figsize=(10,6))
plt.title(f"Clustering of communities with increasing $\epsilon$ for $p = {p}$")
plt.grid()
plt.legend(loc = 'best')

# plt.scatter(x_with_testkits1, y_with_testkits1, c='#5dbb63', label='With Testkits', s=1)
# plt.scatter(x_without_testkits1, y_without_testkits1, c='#dc143c', label='Without Testkit', s=1)

plt.fill_between(epsilon_list, y_with_testkits1_min, y_with_testkits1_max, color='#90ee90', label='With testkits (denoised)', alpha=0.5)
plt.fill_between(epsilon_list, y_without_testkits1_min, y_without_testkits1_max, color='#ff8a8a', label='Without testkits (denoised)', alpha=0.5)

plt.plot(epsilon_list, y_with_testkits1_mean, c='#013220', label='Mean of $I_{max}$ in nodes with testkits', linestyle = 'solid')
plt.plot(epsilon_list, y_without_testkits1_mean, c='#750000', label='Mean of $I_{max}$ in nodes without testkits', linestyle = 'solid')

# Plotting for reduced equations
plt.plot(epsilon_list, x_with_testkitsR1, c='#4285F4', label='Mean of $I_{max}$ in nodes with testkits', linestyle = 'solid')
plt.plot(epsilon_list, x_without_testkitsR1, c='#4285F4', label='Mean of $I_{max}$ in nodes without testkits', linestyle = 'solid')


plt.xlabel("$\log{\epsilon}$")
plt.ylabel("$I_{max}$")

plt.xscale('log')

plt.show()

In [ ]:
# X and Y with testkits
x_with_testkits2 = np.array([])
y_with_testkits2 = np.array([])
y_with_testkits2_mean = np.array([])
y_with_testkits2_max = np.array([])
y_with_testkits2_min = np.array([])

# X and Y without testkits
x_without_testkits2 = np.array([])
y_without_testkits2 = np.array([])
y_without_testkits2_mean = np.array([])
y_without_testkits2_max = np.array([])
y_without_testkits2_min = np.array([])

# Arrays for reduced equation solver
x_with_testkitsR2 = np.array([])
x_without_testkitsR2 = np.array([])
    
for epsilon_value in epsilon_list:
    # initial conditions
    custom_E0 = np.random.randint(1, max_exposed, size=(M,1))
#------------------Reduced Model-----------------------
    E1,E2= custom_E0[0][0],custom_E0[l2][0]
    # Initial conditions for given epsilon
    X0= np.array([N-E1,E1,0,0,N-E2,E2,0,0,0,0,0])

    X_flat = X0.ravel()

    # Solving the initial value problem
    solved_ivp = solve_ivp(ODE_for_reduced_eqns, (t0, tf), X_flat, args=(l2, M-l2, epsilon_value,  0.03, gamma, sigma, zeta, chi, a0, a1))

    # Obtaining the Infection values for both communities
    I_1 = solved_ivp.y[2]
    I_2 = solved_ivp.y[6]

    # Getting the I_max values from I_vals
    I_max_1 = np.max(I_1)
    I_max_2 = np.max(I_2)
    
    x_with_testkitsR2 = np.append(x_with_testkitsR2, I_max_1/N)
    x_without_testkitsR2 = np.append(x_without_testkitsR2, I_max_2/N)
#--------------------------------------------------------

    # Solving the IVP using our solver function
    solved_ivp = solver(adj_mat, adj_mat_normalised, degree_list, M, 0.03, ODE_multi_node_with_testkit, custom_E0, epsilon_value, M, l2)

    # Obtaining the shape we need to realize the y values
    reshape_vals = (6, M, solved_ivp.y.shape[1])
    
    # Getting the y value reshaped
    jffy = solved_ivp.y.reshape(reshape_vals)[2]

    # Obtaining the Infection values for each communities
    I_vals = (solved_ivp.y.reshape(6, M, reshape_vals[2])[2]).reshape(M, reshape_vals[2])

    # Getting the I_max values from I_vals
    I_max = np.max(I_vals, axis=1).reshape(M, 1)
    
    # Colour list for testkit and testkit-free communities
    color_list = ["#F08080", "#80F080", "#8080F0"]
    
    # x1, y1 for the "with testkit nodes"
    x1 = np.full(M, epsilon_value)[:l2]
    y1 = I_max.T[0][:l2] / N
    marker_color_with_testkits = color_list[0]
    
    y_with_testkits2 = np.append(y_with_testkits2, y1)
    x_with_testkits2 = np.append(x_with_testkits2, x1)
    y_with_testkits2_mean = np.append(y_with_testkits2_mean, [np.sum(y1) / l2])
    y_with_testkits2_max = np.append(y_with_testkits2_max, [np.max(y1)])
    y_with_testkits2_min = np.append(y_with_testkits2_min, [np.min(y1)])
    
    # x2, y2 for the "without testkit nodes"
    x2 = np.full(M, epsilon_value)[l2:-1]
    y2 = I_max.T[0][l2:-1] / N
    marker_color_without_testkits = color_list[1]
    
    y_without_testkits2 = np.append(y_without_testkits2, y2)
    x_without_testkits2 = np.append(x_without_testkits2, x2)
    y_without_testkits2_mean = np.append(y_without_testkits2_mean, [np.sum(y2) / (y2.shape[0])])
    y_without_testkits2_max = np.append(y_without_testkits2_max, [np.max(y2)])
    y_without_testkits2_min = np.append(y_without_testkits2_min, [np.min(y2)])
    
    # Increment the current epsilon step number
    I_max_vs_epsilon_list = np.zeros(shape=(epsilon_steps, M))

In [ ]:
plt.figure(figsize=(10,6))
plt.title(f"Clustering of communities with increasing $\epsilon$ for $p = {1. - p}$")
plt.grid()
plt.legend(loc = 'best')

# plt.scatter(x_with_testkits2, y_with_testkits2, c='#5dbb63', label='With Testkits', s=1)
# plt.scatter(x_without_testkits2, y_without_testkits2, c='#dc143c', label='Without Testkit', s=1)

plt.fill_between(epsilon_list, y_with_testkits2_min, y_with_testkits2_max, color='#90ee90', label='With testkits (denoised)', alpha=0.5)
plt.fill_between(epsilon_list, y_without_testkits2_min, y_without_testkits2_max, color='#ff8a8a', label='Without testkits (denoised)', alpha=0.5)

plt.plot(epsilon_list, y_with_testkits2_mean, c='#013220', label='Mean of $I_{max}$ in nodes with testkits', linestyle = 'solid')
plt.plot(epsilon_list, y_without_testkits2_mean, c='#750000', label='Mean of $I_{max}$ in nodes without testkits', linestyle = 'solid')

# Plotting for reduced equations
plt.plot(epsilon_list, x_with_testkitsR2, c='#4285F4', label='Mean of $I_{max}$ in nodes with testkits', linestyle = 'solid')
plt.plot(epsilon_list, x_without_testkitsR2, c='#4285F4', label='Mean of $I_{max}$ in nodes without testkits', linestyle = 'solid')


plt.xlabel("$\log{\epsilon}$")
plt.ylabel("$I_{max}$")

plt.xscale('log')

plt.show()

In [ ]:
with plt.style.context('seaborn-paper'):
    # Making the figure and axes
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(38, 10))
    
    # Font sizes for everything
    fs_label = 44
    fs_ticks = 32
    fs_title = 34
    
    ###############################################################
    # Subplot 1
    axs[0].fill_between(epsilon_list, y_with_testkits1_min, y_with_testkits1_max, color='#90ee90', alpha=0.5)
    axs[0].fill_between(epsilon_list, y_without_testkits1_min, y_without_testkits1_max, color='#ff8a8a', alpha=0.5)

    axs[0].plot(epsilon_list, y_with_testkits1_mean, c='#013220', label='Mean of $I_{max}$ in nodes with testkits', linestyle = '-.', linewidth=3)
    axs[0].plot(epsilon_list, y_without_testkits1_mean, c='#750000', label='Mean of $I_{max}$ in nodes without testkits', linestyle = '-.', linewidth=3)
    
    # Plotting for reduced equations
    axs[0].plot(epsilon_list, x_with_testkitsR1,'ob-', label='$I$ in testkit node in reduced eqns.')
    axs[0].plot(epsilon_list, x_without_testkitsR1, 'ob-', label='$I$ in non-testkit node in reduced eqns.')

    axs[0].set_xlabel("$\log{\epsilon}$", fontsize=fs_label)
    axs[0].set_ylabel("$I_{max}$", fontsize=fs_label)
    
    axs[0].set_xscale('log')
    
    axs[0].set_title(f"Clustering of communities with increasing $\epsilon$ for $p = {p}$", fontsize=fs_title)
    
    ###############################################################
    # Subplot 2
    axs[1].fill_between(epsilon_list, y_with_testkits2_min, y_with_testkits2_max, color='#90ee90', alpha=0.5)
    axs[1].fill_between(epsilon_list, y_without_testkits2_min, y_without_testkits2_max, color='#ff8a8a', alpha=0.5)

    axs[1].plot(epsilon_list, y_with_testkits2_mean, c='#013220', label='Mean of $I_{max}$ in nodes with testkits', linestyle = '-.', linewidth=3)
    axs[1].plot(epsilon_list, y_without_testkits2_mean, c='#750000', label='Mean of $I_{max}$ in nodes without testkits', linestyle = '-.', linewidth=3)
    
    # Plotting for reduced equations
    axs[1].plot(epsilon_list, x_with_testkitsR2,'ob-', label='$I$ in testkit node in reduced eqns.')
    axs[1].plot(epsilon_list, x_without_testkitsR2,'ob-', label='$I$ in non-testkit node in reduced eqns.')

    axs[1].set_xlabel("$\log{\epsilon}$", fontsize=fs_label)
    axs[1].set_ylabel("$I_{max}$", fontsize=fs_label)

    axs[1].set_xscale('log')
    
    axs[1].set_title(f"Clustering of communities with increasing $\epsilon$ for $p = {1. - p}$", fontsize=fs_title)
    
    ###############################################################
    
    # Adding xlabels, legends, logscale(x), and grid
    
    for j in range(2):
        axs[j].set_xlabel("$\epsilon$", fontsize = fs_label)
#         axs[j].grid()
#         axs[j].legend(loc = 'upper right', prop = {'size' : fs_legend})
        axs[j].set_xscale('log')

        axs[j].text(
                    -0.1, 
                    1.1, 
                    string.ascii_uppercase[j], 
                    transform=axs[j].transAxes, 
                    size=35, 
                    weight='bold'
                    )

        axs[j].tick_params(direction= 'in', which='major', length=7,width=3)
        axs[j].tick_params(direction= 'in', which='minor', length=5, width=2)

        for axis in ['top','bottom','left','right']:
            axs[j].spines[axis].set_linewidth(3)

        x_ticks_labels = [r'$10^{-5}$', r'$10^{-3}$', r'$10^{-1}$', r'$10^{1}$']
        x_ticks_vals = [10**(-5), 10**(-3), 10**(-1), 10**(1)]

        axs[j].set_xticks([float(x_tick) for x_tick in x_ticks_vals],
                         [str(x_tick) for x_tick in x_ticks_labels],
                            fontsize=fs_ticks)

        axs[j].set_xlim(epsilon_list[0], epsilon_list[-1])

        y_ticks_imax = [0, 0.1, 0.2, 0.3]
        axs[j].set_yticks([float(y_tick) for y_tick in y_ticks_imax],
                         [str(round(y_tick, 2)) for y_tick in y_ticks_imax],
                        fontsize=fs_ticks)
            
    # Adding ylabels

    axs[0].set_ylabel("$I_{max}$")

    axs[0].set_ylim([-0.015,0.3+0.015])
    axs[0].set_title(f"Clustering of communities with increasing $\epsilon$ for $p = {p}$", fontsize=fs_title)

    axs[1].set_ylabel("$I_{max}$")

    axs[1].set_ylim([-0.015,0.3+0.015])
    axs[1].set_title(f"Clustering of communities with increasing $\epsilon$ for $p = {float(1) - p}$", fontsize=fs_title)
    
    labels_handles = {
        label: handle for ax in fig.axes for handle, label in zip(*ax.get_legend_handles_labels())
    }
    
    fs_legend= fs_title
    
    fig.legend(
        labels_handles.values(),
        labels_handles.keys(),
        loc="lower center",  # Change the location to "lower center" for bottom placement
        ncol=2,  # Display the legend in two columns
        bbox_to_anchor=(0.5, 1.05),  # Adjust the Y value to move the legend to the bottom
        bbox_transform=plt.gcf().transFigure,
        fontsize=fs_legend,  # Set the font size for the legend
    )
    fig.savefig('complete_red_model.pdf', dpi=1200, bbox_inches='tight')